## Import modules

In [ ]:
import pdb
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
plt.style.use('ggplot')

import nhanes as nhanes

%matplotlib notebook

## Settings

In [ ]:
DATA_PATH = 'CDC/NHANES/'
DATASET = 'cancer'

### Note: 
The code below loads each dataset: dataset_features, dataset_targets

Here, all datasets are defined explicitly (see nhanes.py).

In [ ]:
ds = nhanes.Dataset(DATA_PATH)
ds.load_arthritis()
n_fe = ds.features.shape[1]
n_classes = 2

dataset_features = ds.features
dataset_targets = ds.targets

## Train/Test Separation

In [ ]:
perm = np.random.permutation(dataset_targets.shape[0])
dataset_features = dataset_features[perm]
dataset_targets = dataset_targets[perm]

def get_batch(n_size, phase):
    # select indices
    n_samples = dataset_features.shape[0]
    n_classes = int(dataset_targets.max() + 1)
    if phase == 'test':
        inds_sel = np.arange(0, int(n_samples*0.15), 1)
    elif phase == 'validation':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.15), int(n_samples*0.30), 1)
    elif phase == 'train':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.30), n_samples, 1)
    else:
        raise NotImplementedError
    inds_sel = np.random.permutation(inds_sel)
    batch_inds = []
    for cl in range(n_classes):
        inds_cl = inds_sel[dataset_targets[inds_sel] == cl]
        batch_inds.extend(inds_cl[:n_size//n_classes])
    batch_inds = np.random.permutation(batch_inds)
    
    return dataset_features[batch_inds], dataset_targets[batch_inds]
    
features_trn, targets_trn = get_batch(n_size=8000, phase='train')
features_tst, targets_tst = get_batch(n_size=2000, phase='test')

## Classification

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_RFC', accu)

clf = SVC(gamma='auto')
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_SVC', accu)

clf = LogisticRegression(solver='lbfgs', max_iter=200)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_LR', accu)


#### Baseline Performance Function

In [ ]:
def baseline_perf(df,y):
    rf = RandomForestClassifier(n_estimators=100)
    rf_score = cross_val_score(rf,df,y,cv=5)
    
    svm = SVC(gamma='auto')
    svm_score = cross_val_score(svm,df,y,cv=5)
    
    lr = LogisticRegression(solver='lbfgs',max_iter=200)
    lr_score = cross_val_score(lr,df,y,cv=5)
    
    return [rf_score.mean(),svm_score.mean(),lr_score.mean()]

## Feature Selection/Engineering

#### Remove correlated variables

In [ ]:
X_train = pd.DataFrame(features_trn)
X_test = pd.DataFrame(features_tst)

In [ ]:
baseline_perf(X_train,y_train)

In [ ]:
X_corr = X_train.corr()

In [ ]:
plt.figure(figsize=(10,5))

sns.heatmap(X_corr, cmap='coolwarm',xticklabels=False,yticklabels=False)

In [ ]:
to_drop = []
for i in range(len(X_corr)):
    for j in range(len(X_corr)):
        if (i != j) and (X_corr.iloc[i,j]>0.7):
            if ([i,j] not in to_drop) and ([j,i] not in to_drop):
                to_drop.append([i,j])

idxs = []
for i in to_drop:
    if (i[0] not in idxs) and (i[1] not in idxs):
        idxs.append(i[1])

In [ ]:
X_train.drop(idxs,axis=1,inplace=True)
X_test.drop(idxs,axis=1,inplace=True)

In [ ]:
baseline_perf(X_train,y_train)

In [ ]:
X_corr_post = X_train.corr()

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
sns.heatmap(X_corr, cmap='coolwarm',xticklabels=False,yticklabels=False)
plt.title('Pre-Filter')

plt.subplot(1,2,2)
sns.heatmap(X_corr_post, cmap='coolwarm',xticklabels=False,yticklabels=False)
plt.title('Post-Filter')

plt.savefig('corrfilter.jpg')

#### Mutual Information filter

In [ ]:
y_train = pd.Series(targets_trn)

In [ ]:
mi = mutual_info_classif(X_train,y_train)

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.hist(mi,bins=50)
plt.xlabel('Mutual Information')
plt.ylabel('# of Features')


plt.subplot(1,2,2)
sns.boxplot(x=mi)
plt.xlabel('Mutual Information')

plt.savefig('mi.jpg')

In [ ]:
mi_cut = np.percentile(mi,25)
to_keep = (mi > mi_cut)

In [ ]:
X_train = X_train.loc[:,to_keep]
X_test = X_test.loc[:,to_keep]

In [ ]:
baseline_perf(X_train,y_train)

#### Variance filter

In [ ]:
bin_idxs = []
cont_idxs = []
for i in X_train.columns:
    if X_train.loc[:,i].nunique() <= 2:
        bin_idxs.append(i)
    else:
        cont_idxs.append(i)

In [ ]:
X_train_bin = X_train.loc[:,bin_idxs]
X_train_cont = X_train.loc[:,cont_idxs]

In [ ]:
bin_var = X_train_bin.var() 

In [ ]:
bin_cut = np.percentile(bin_var,25)

In [ ]:
X_train_bin = X_train_bin.loc[:,(bin_var > bin_cut)]

In [ ]:
cont_var = X_train_cont.var()

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,4,1)
plt.hist(bin_var,bins=50)
plt.xlabel('Variance')
plt.ylabel('# of Features')
plt.title('Categorical')

plt.subplot(1,4,2)
sns.boxplot(x=bin_var)
plt.xlabel('Variance')
plt.title('Categorical')

plt.subplot(1,4,3)
plt.hist(cont_var,bins=10)
plt.xlabel('Variance')
plt.ylabel('# of Features')
plt.title('Continuous')


plt.subplot(1,4,4)
sns.boxplot(x=cont_var)
plt.xlabel('Variance')
plt.title('Continuous')


plt.savefig('varfil.jpg')

In [ ]:
cont_cut = np.percentile(cont_var,25)

In [ ]:
X_train_cont = X_train_cont.loc[:,(cont_var > cont_cut)]

In [ ]:
X_train = pd.concat([X_train_bin,X_train_cont],axis=1)

In [ ]:
X_test = X_test.loc[:,X_train.columns]

In [ ]:
baseline_perf(X_train,y_train)

#### Logistic Regression for Feature Selection

In [ ]:
C = np.logspace(-4,4,20)
scores = []
for i in C:
    print(i)
    lrfs = LogisticRegression(solver='liblinear',penalty='l1',C=i)
    cvs = cross_val_score(lrfs,X_train,y_train, cv=5)
    scores.append(cvs.mean())

In [ ]:
plt.figure(figsize=(10,5))

plt.plot(np.log(C),scores,marker='o',color='blue')

plt.xlabel('Regularization Coefficient (Log Scale)')
plt.ylabel('C-V Accuracy')
plt.title('Lasso Optimization')
plt.savefig('lasso.jpg')

In [ ]:
lrfs = LogisticRegression(penalty='l1',C=C[7])
lrfs.fit(X_train,targets_trn)

In [ ]:
X_train = X_train.loc[:,(lrfs.coef_ > 0)[0]]
X_test = X_test.loc[:,(lrfs.coef_ > 0)[0]]

#### Polynomial Features

In [ ]:
poly = PolynomialFeatures(2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
X_train_poly = pd.DataFrame(X_train_poly)
X_test_poly = pd.DataFrame(X_test_poly)

In [ ]:
baseline_perf(X_train_poly,y_train)

In [ ]:
mi_poly = mutual_info_classif(X_train_poly,y_train)

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.hist(mi_poly,bins=50)
plt.xlabel('Mutual Information of Polynomial Features')
plt.ylabel('# of Features')


plt.subplot(1,2,2)
sns.boxplot(x=mi_poly)
plt.xlabel('Mutual Information of Polynomial Features')

#plt.savefig('mi.jpg')

In [ ]:
mi_cut = np.percentile(mi_poly,90)
to_keep = (mi_poly > mi_cut)

In [ ]:
to_keep.sum()

In [ ]:
X_train_poly = X_train_poly.loc[:,to_keep]
X_test_poly = X_test_poly.loc[:,to_keep]

In [ ]:
cut = np.percentile(X_train_poly.var(),90)
X_train_poly = X_train_poly.loc[:,(X_train_poly.var() > cut)]
X_test_poly = X_test_poly.loc[:,(X_train_poly.var() > cut)]

In [ ]:
baseline_perf(X_train_poly,y_train)

#### PCA

In [ ]:
cvs_pca = []
n = range(5,51,5)
for i in n:
    print(i)
    pca_ = PCA(n_components=i)
    X_pca = pca_.fit_transform(X_train)
    gbdt_pca = GradientBoostingClassifier(n_estimators=50,min_samples_split=0.5)
    cvs = cross_val_score(gbdt_pca,X_pca,y_train, cv=5)
    cvs_pca.append(cvs.mean())

print(cvs_pca)    

## Model Optimizations

#### Decision Trees

In [ ]:
cv = []
for i in range(1,21):
    dt = DecisionTreeClassifier(max_depth=i)
    cvs = cross_val_score(dt,X_train,y_train, cv=5)
    cv.append(cvs.mean())

In [ ]:
plt.figure(figsize=(10,5))

plt.plot(list(range(1,21)),cv,marker='o',color='blue')

plt.xlabel('DTree Depth')
plt.ylabel('Accuracy')
plt.title('Decision Tree Optimization')
plt.savefig('Dtree_opt.jpg')

#### Support Vector Classifier

In [ ]:
C = [0.1,1,10]
kernel = ['linear','rbf']

In [ ]:
max_cv = 0
combo = None
for i in C:
    for j in kernel:
        print([i,j])
        svc = SVC(C=i,kernel=j)
        cvs = cross_val_score(svc,X_train,y_train, cv=5)
        curr_cv = cvs.mean()
        print(curr_cv)
        if curr_cv > max_cv:
            max_cv = curr_cv
            combo = [i,j]
                

#### Random Forests

In [ ]:
n_est = [50,100,150]
n_samp = [0.1,0.3,0.5,0.7]
n_feat = [0.1,0.3,0.5,0.7]

In [ ]:
max_cv = 0
combo = None
for i in n_est:
    for j in n_samp:
        for k in n_feat:
            print([i,j,k])
            rf = RandomForestClassifier(n_estimators=i,min_samples_split=j,max_features=k)
            cvs = cross_val_score(rf,X_train,y_train, cv=5)
            curr_cv = cvs.mean()
            print(curr_cv)
            if curr_cv > max_cv:
                max_cv = curr_cv
                combo = [i,j,k]

In [ ]:
n_est = [60]
n_samp = np.array((range(5,16,1)))/100
n_feat = np.array((range(5,16,1)))/100

rf_60 = pd.DataFrame(data=np.zeros((11,11)),index=n_samp,columns=n_feat)
for i in n_est:
    for j in n_samp:
        for k in n_feat:
            rf = RandomForestClassifier(n_estimators=i,min_samples_split=j,max_features=k)
            cvs = cross_val_score(rf,X_train,y_train, cv=5)
            curr_cv = cvs.mean()
            rf_60.loc[j,k] = curr_cv


In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
sns.heatmap(rf_40,cbar=False,cmap='coolwarm')
plt.xlabel('Feat %')
plt.ylabel('Samp %')
plt.title('N_est = 40')

plt.subplot(1,3,2)
sns.heatmap(rf_50,cbar=False,yticklabels=False,cmap='coolwarm')
plt.xlabel('Feat %')
plt.title('N_est = 50')

plt.subplot(1,3,3)
sns.heatmap(rf_60,yticklabels=False,cmap='coolwarm')
plt.xlabel('Feat %')
plt.title('N_est = 60')

plt.tight_layout()
plt.savefig('rfopt.jpg')

#### Gradient Boosted Decision Trees

In [ ]:
n_est = [150]
n_samp = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

cv150 = []
for i in n_est:
    for j in n_samp:
        print([i,j])
        gbdt = GradientBoostingClassifier(n_estimators=i,min_samples_split=j)
        cvs = cross_val_score(gbdt,X_train,y_train, cv=5)
        curr_cv = cvs.mean()
        cv150.append(curr_cv)

In [ ]:
n_est = [70]
n_samp = np.array(range(5,16,1))/100

cv70 = []
for i in n_est:
    for j in n_samp:
        print([i,j])
        gbdt = GradientBoostingClassifier(n_estimators=i,min_samples_split=j)
        cvs = cross_val_score(gbdt,X_train,y_train, cv=5)
        curr_cv = cvs.mean()
        cv70.append(curr_cv)

In [ ]:
plt.figure(figsize=(10,5))

plt.plot(n_samp,cv30,marker='o',color='red',label='30 ests')
plt.plot(n_samp,cv40,marker='o',color='orange',label='40 ests')
plt.plot(n_samp,cv50,marker='o',color='yellow',label='50 ests')
plt.plot(n_samp,cv60,marker='o',color='green',label='60 ests')
plt.plot(n_samp,cv70,marker='o',color='blue',label='70 ests')

plt.xlabel('Sample %')
plt.ylabel('Accuracy')
plt.title('Gradient-Boosted Decision Tree Optimization')
plt.legend()
plt.tight_layout()
plt.savefig('gbdtopt.jpg')

In [ ]:
gbdt = GradientBoostingClassifier(n_estimators=30,min_samples_split=0.2)
gbdt.fit(X_train,y_train)

In [ ]:
fi = pd.Series(data = gbdt.feature_importances_, index=X_train.columns)

In [ ]:
top_fi = fi.sort_values(ascending=False).head(10)

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=top_fi.values,y=top_fi.index,orient='h',order=top_fi.index)

plt.xlabel('Feature Importance')
plt.ylabel('Feature Index')
plt.title('GBDT Feature Importances')

plt.savefig('topfi.jpg')

##### Final Predictions

In [ ]:
df = pd.DataFrame(dataset_features)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(df, dataset_targets, test_size=0.33, random_state=42)

In [ ]:
X_te.columns = [str(i) for i in X_te.columns]

In [ ]:
X_te = X_te.loc[:,X_train.columns]

In [ ]:
final_preds = gbdt.predict(X_te)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print(confusion_matrix(y_te,final_preds))
print('\n')
print(accuracy_score(y_te,final_preds))

In [ ]:
X_tr.columns = [str(i) for i in X_tr.columns]
X_tr = X_tr.loc[:,X_train.columns]

In [ ]:
gbdt2 = GradientBoostingClassifier(n_estimators=30,min_samples_split=0.2)
gbdt2.fit(X_tr,y_tr)
preds2 = gbdt2.predict(X_te)

In [ ]:
print(confusion_matrix(y_te,preds2))
print('\n')
print(accuracy_score(y_te,preds2))

#### Densely Connected Neural Network

In [ ]:
import tensorflow as tf

In [ ]:
import random 
def partition (list_in, n):
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]

In [ ]:
#Create folds
idx_folds = partition(list(X_train.index),5)

tr1 = idx_folds[0] + idx_folds[1] + idx_folds[2] + idx_folds[3]
tr2 = idx_folds[0] + idx_folds[2] + idx_folds[3] + idx_folds[4]
tr3 = idx_folds[0] + idx_folds[1] + idx_folds[3] + idx_folds[4]
tr4 = idx_folds[0] + idx_folds[1] + idx_folds[2] + idx_folds[4]
tr5 = idx_folds[1] + idx_folds[2] + idx_folds[3] + idx_folds[4]
tr = [tr1,tr2,tr3,tr4,tr5]

te1 = idx_folds[4]
te2 = idx_folds[1]
te3 = idx_folds[2]
te4 = idx_folds[3]
te5 = idx_folds[0]
te = [te1,te2,te3,te4,te5]

In [ ]:
X_train.columns = [str(i) for i in X_train.columns]

In [ ]:
cvs = []
for i in range(len(tr)):
    print(i)
    #Suppress output except for error messages
    tf.logging.set_verbosity(tf.logging.ERROR)

    #feature columns
    feature_columns = []
    for j in X_train.columns:
        feature_columns.append(tf.feature_column.numeric_column(str(j)))

    #Model Initialization
    classifier = tf.estimator.DNNClassifier(
        feature_columns=feature_columns,
        hidden_units=[100, 50, 25],
        optimizer=tf.train.AdamOptimizer(1e-2),
        n_classes=2,
        dropout=0.2,
    )

    #Input Function
    train_input_fn = tf.estimator.inputs.pandas_input_fn(
        x=X_train.iloc[tr[i],:],
        y=y_train.iloc[tr[i]],
        num_epochs=50,
        batch_size=len(tr[i]),
        shuffle=True
    )

    #model training
    classifier.train(input_fn=train_input_fn, steps=10000)

    #eval function
    test_input_fn = tf.estimator.inputs.pandas_input_fn(
        x=X_train.iloc[te[i],:],
        y=y_train.iloc[te[i]],
        num_epochs=1,
        shuffle=False
    )

    #predictions
    preds = list(classifier.predict(test_input_fn))

    pred_class = [p["classes"] for p in preds]

    preds = []

    for k in range(len(pred_class)):
        preds.append(int(pred_class[k][0]))
    
    acc = accuracy_score(y_train.iloc[te[i]],preds)
    print(acc)
    
    cvs.append(acc)

In [ ]:
np.mean(cvs)